# Ladataan englanti-suomi lauseparit tiedostosta
# Lisätään [start] ja [end] -tokenit dekooderin harjoittelua varten

In [30]:
text_file = "fin-eng/fin.txt"

with open(text_file, encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
    english, finnish, _ = line.split("\t")
    finnish = "[start] " + finnish + " [end]"
    text_pairs.append((english, finnish))


# Tuodaan tarvittavat kirjastot ja sekoitetaan lauseet
# Jaetaan 85 % koulutusdataan ja 15 % validointiin
# Erotellaan lähdekieli (englanti) ja kohdekieli (suomi)
# TextVectorization muuntaa tekstin kokonaislukujonoiksi.
# Mahdollistaa sanojen syöttämisen neuroverkkomalliin.
# Käytetään 15 000 sanaa ja maksimipituus 40 sanaa

In [31]:
import random
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras import layers, models
from tensorflow import keras


random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
train_pairs = text_pairs[:-num_val_samples]
val_pairs = text_pairs[-num_val_samples:]

source_texts = [pair[0] for pair in train_pairs]
target_texts = [pair[1] for pair in train_pairs]

source_vectorization = TextVectorization(max_tokens=15000, output_mode="int", output_sequence_length=40)
target_vectorization = TextVectorization(max_tokens=15000, output_mode="int", output_sequence_length=40)

source_vectorization.adapt(source_texts)
target_vectorization.adapt(target_texts)

# Muotoillaan data Transformer-mallia varten.
# Palauttaa syötteet kooderille ja dekooderille + oikeat vastaukset.
# Tehdään TensorFlow-datasetit harjoitteluun ja validointiin.


In [32]:
def format_dataset(src, tgt):
    src = source_vectorization(src)
    tgt = target_vectorization(tgt)
    return ({"encoder_inputs": src, "decoder_inputs": tgt[:, :-1]}, tgt[:, 1:])

batch_size = 64
train_ds = tf.data.Dataset.from_tensor_slices((source_texts, target_texts))
train_ds = train_ds.batch(batch_size).map(format_dataset).prefetch(1)

val_source_texts = [pair[0] for pair in val_pairs]
val_target_texts = [pair[1] for pair in val_pairs]

val_ds = tf.data.Dataset.from_tensor_slices((val_source_texts, val_target_texts))
val_ds = val_ds.batch(batch_size).map(format_dataset).prefetch(1)

# PositionalEmbedding lisää sanojen järjestystiedon mallille.
# Transformer tarvitsee tämän ymmärtääkseen sanojen sijainnin lauseessa.


In [33]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return keras.ops.not_equal(inputs, 0)


    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config

# TransformerEncoder hyödyntää monipäistä huomiomekanismia ja tiheää verkkoa.
# Koodaa syötelauseen numeeriseksi esitykseksi.


In [34]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.attention.key_dim,
            "dense_dim": self.dense_proj.layers[0].units,
            "num_heads": self.attention.num_heads,
        })
        return config

# TransformerDecoder käyttää myös monipäistä huomiota ja kausaalimaskia.
# Mahdollistaa sanan ennustamisen yksi kerrallaan ilman tulevien sanojen näkemistä.


In [35]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(latent_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(inputs, inputs, attention_mask=causal_mask)
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(out_1, encoder_outputs, attention_mask=padding_mask)
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)


    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, seq_length = input_shape[0], input_shape[1]
        i = tf.range(seq_length)[:, tf.newaxis]
        j = tf.range(seq_length)
        mask = tf.cast(i >= j, dtype="int32")
        return tf.reshape(mask, (1, seq_length, seq_length))

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.attention_1.key_dim,
            "latent_dim": self.dense_proj.layers[0].units,
            "num_heads": self.attention_1.num_heads,
        })
        return config


# Rakennetaan koko malli: syöte kooderille → tulos dekooderille.
# Lopputulos on sanan todennäköisyysjakauma seuraavaksi sanaksi.
# Käytetään sparse_categorical_crossentropy -häviötä koska vastaukset ovat kokonaislukuja.


In [36]:
embed_dim = 256
dense_dim = 1024
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(40, 15000, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
x = PositionalEmbedding(40, 15000, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
decoder_outputs = layers.Dense(15000, activation="softmax")(x)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,850,240 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, None)      │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,850,240 │ decoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 256) │  2,630,144 │ positional_embed… │
│ (TransformerEncode… │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, None, 256) │  4,734,208 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, None,      │  3,855,000 │ transformer_deco… │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 18,919,832 (72.17 MB)

 Trainable params: 18,919,832 (72.17 MB)

 Non-trainable params: 0 (0.00 B)

# Harjoitetaan mallia 3 epochin ajan.
# Tämä riittää osoittamaan, että malli oppii – tehtävä ei vaadi täydellistä käännöstä.
# Arvioidaan malli harjoitusdatalla.
# Antaa käsityksen siitä, kuinka hyvin malli oppi.


In [37]:
history = model.fit(train_ds, epochs=3, validation_data=val_ds)

model.evaluate(train_ds)

Epoch 1/3
960/960 ━━━━━━━━━━━━━━━━━━━━ 931s 965ms/step - accuracy: 0.0756 - loss: 4.6717 - val_accuracy: 0.0944 - val_loss: 2.8368
Epoch 2/3
960/960 ━━━━━━━━━━━━━━━━━━━━ 922s 961ms/step - accuracy: 0.0985 - loss: 2.6848 - val_accuracy: 0.1039 - val_loss: 2.2489
Epoch 3/3
960/960 ━━━━━━━━━━━━━━━━━━━━ 915s 953ms/step - accuracy: 0.1086 - loss: 2.0269 - val_accuracy: 0.1059 - val_loss: 2.1272
960/960 ━━━━━━━━━━━━━━━━━━━━ 251s 261ms/step - accuracy: 0.1127 - loss: 1.7775


[1.653383493423462, 0.11564385890960693]

# Tämä funktio tekee käännöksen annetusta englanninkielisestä lauseesta suomeksi.
# Ennustaa seuraavan sanan yksi kerrallaan, kunnes pääsee [end]-tokeniin.
# Testataan mallia yhdellä esimerkkilauseella.
# Tulostetaan mallin tuottama suomennos.


In [38]:
def decode_sequence(input_sentence):
    tokenized_input = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for _ in range(40):
        tokenized_target = target_vectorization([decoded_sentence])[:, :-1]
        predictions = model.predict({"encoder_inputs": tokenized_input, "decoder_inputs": tokenized_target}, verbose=0)
        sampled_token_index = np.argmax(predictions[0, -1, :])
        sampled_token = target_vectorization.get_vocabulary()[sampled_token_index]
        if sampled_token == "[end]":
            break
        decoded_sentence += " " + sampled_token
    return decoded_sentence

# Test
print(decode_sequence("I am hungry."))


[start] end end end end end minulla minulla minulla minulla minulla nälkä nälkä end nälkä minulla nälkä nälkä nälkä end nälkä nälkä end nälkä minulla nälkä nälkä end nälkä nälkä on nälkä end nälkä nälkä end nälkä nälkä on nälkä nälkä
